# Criando Dimensão Tipo de envio
- scd 1

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


In [2]:
# Importando as bibliotecas
import pandas as pd 
from sqlalchemy import text
from sqlalchemy import create_engine
from dotenv import load_dotenv
from utils.acessobanco import acessobanco
from datetime import datetime


In [3]:
# Carrega as variáveis de ambiente do arquivo .env

load_dotenv(dotenv_path="/home/danielpedro/Python/Projetos/SuperStorePY/Bancos/acessobanco.env")


True

In [4]:
# Cria a conexão com o banco usando a função acessobanco()
engine = acessobanco()

In [5]:
# Criando a df puxando do banco
df = pd.read_sql_table("orders",engine, schema="stg")

In [6]:
# Verificando as colunas e excluindo os tipos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             9994 non-null   object
 1   id_pedido      9994 non-null   object
 2   data_pedido    9994 non-null   object
 3   data_envio     9994 non-null   object
 4   modo_envio     9994 non-null   object
 5   id_cliente     9994 non-null   object
 6   nome_cliente   9994 non-null   object
 7   segmento       9994 non-null   object
 8   pais           9994 non-null   object
 9   cidade         9994 non-null   object
 10  estado         9994 non-null   object
 11  ce             9994 non-null   object
 12  regiao         9994 non-null   object
 13  id_produto     9994 non-null   object
 14  categoria      9994 non-null   object
 15  sub_categoria  9994 non-null   object
 16  nome_produto   9994 non-null   object
 17  valor_venda    9994 non-null   object
 18  quantidade     9994 non-null

In [7]:
# Criando df tipo envio
df_TipoEnvio =  df[['modo_envio']]

In [8]:
# Verificar os dados duplicados
df_TipoEnvio.duplicated().value_counts()

True     9990
False       4
Name: count, dtype: int64

In [9]:
# Excluir os dados duplicados
df_TipoEnvio = df_TipoEnvio.drop_duplicates(subset='modo_envio')

In [10]:
# Verificando se tem duplicados
df_TipoEnvio.duplicated().value_counts()

False    4
Name: count, dtype: int64

In [11]:
# vizualizando a tabela
df_TipoEnvio.head()

,modo_envio
0,Standard Class
9,First Class
14,Second Class
38,Same Day


In [12]:
# Criando um id para o modo de envio
df_TipoEnvio['id_modo_envio'] = range(1,len(df_TipoEnvio) + 1)

In [13]:
#ajustando o posicionamento das colunas
df_TipoEnvio = df_TipoEnvio[['id_modo_envio', 'modo_envio']]


In [14]:
# Verificando o tipo
df_TipoEnvio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 0 to 38
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_modo_envio  4 non-null      int64 
 1   modo_envio     4 non-null      object
dtypes: int64(1), object(1)
memory usage: 96.0+ bytes


In [15]:
# Criando uma tabela temporaria

with engine.begin() as conn:
    df_TipoEnvio.to_sql(
        "tmp_dim_tipo_envio",  # nome da tabela temporária no schema dw
        conn,               # conexão com o banco de dados
        schema="dw",        # schema onde a tabela será criada
        if_exists="replace", # substitui a tabela se já existir (garante dados atualizados)
        index=False,         # não cria coluna de índice do pandas
        method="multi",      # insere várias linhas por comando INSERT (melhor performance)
        chunksize=5000       # envia os dados em blocos de 5000 linhas (controle de memória)
    )



In [16]:
# para verificarmos quantas linhas subiram

count = pd.read_sql("SELECT COUNT(*) as total_registros FROM dw.tmp_dim_tipo_envio", engine).iloc[0,0]
print(f"Total de registros na tabela: {count}")

Total de registros na tabela: 4


In [17]:
# Insere apenas novo metodos

insert_sql = """
INSERT INTO dw.dim_metodo_envio_scd1 (id_modo_envio, modo_envio)
SELECT t.id_modo_envio, t.modo_envio
FROM dw.tmp_dim_tipo_envio AS t
LEFT JOIN dw.dim_metodo_envio_scd1 AS d ON d.id_modo_envio = t.id_modo_envio
WHERE d.id_modo_envio IS NULL;
"""

with engine.begin() as conn:
    result = conn.execute(text(insert_sql))

print(f"Linhas inseridas: {result.rowcount}")

Linhas inseridas: 0
